In [18]:
# ---------- Central registry of Pauli conjugation rules ----------
# Import Qiskit Library
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, Statevector
import matplotlib.pyplot as plt
from qiskit import __version__ as qiskit_version
import numpy as np
from typing import Callable, Dict, List
from collections import defaultdict

# Print the Qiskit version
print(f"Qiskit version: {qiskit_version}")


# ----------------------------------------------------------------------
# One term  α · P  in a Pauli expansion (Qiskit’s label order)
# ----------------------------------------------------------------------
class PauliPath:
    """
    One term α·P appearing in a Pauli-operator expansion.

    A ``PauliPath`` stores the complex coefficient (``coeff``) and the
    corresponding multi-qubit Pauli operator (``pauli``) expressed as a
    :class:`qiskit.quantum_info.Pauli` object.

    Parameters
    ----------
    coeff : complex
        Complex prefactor multiplying the Pauli operator.
    pauli : qiskit.quantum_info.Pauli
        Tensor product of single-qubit Pauli matrices in Qiskit label order
        (right-most character = qubit-0).

    Attributes
    ----------
    coeff : complex
        Complex prefactor of the Pauli term.
    pauli : qiskit.quantum_info.Pauli
        Underlying Pauli operator.

    Notes
    -----
    * In Qiskit’s little-endian convention, label ``'XYZ'`` represents the
      operator :math:`X \\otimes Y \\otimes Z` acting with *Z* on qubit-0,
      *Y* on qubit-1 and *X* on qubit-2.
    * The helper method :py:meth:`label_lsb` returns the label reversed so
      that the left-most character corresponds to qubit-0 (LSB-first
      representation).
    * The :py:meth:`__repr__` string follows the pattern ``'+0.5·IXY'`` for
      readability in debug output.

    Examples
    --------
    >>> from qiskit.quantum_info import Pauli
    >>> term = PauliPath(0.5, Pauli('IXY'))
    >>> term.coeff
    (0.5+0j)
    >>> term.label_lsb()
    'YXI'
    """
    __slots__ = ("coeff", "pauli")

    def __init__(self, coeff: complex, pauli: Pauli):
        self.coeff = coeff
        self.pauli = pauli

    def label_lsb(self) -> str:
        return self.pauli.to_label()[::-1] 

    def __repr__(self) -> str:                # e.g. "+0.5·IXY"
        # return f"{self.coeff:+g}·{self.pauli.to_label()}"
        return f"{self.coeff:+g}·{self.label_lsb()}"


class QuantumGate:
    """
    A registry for quantum gate Pauli conjugation rules.
    
    This class serves as a central registry for storing and retrieving
    Pauli conjugation rules for different quantum gates. Each rule defines
    how a Pauli operator transforms when conjugated by a specific gate.
    
    Attributes
    ----------
    _registry : Dict[str, Callable]
        Dictionary mapping gate names to their corresponding conjugation rule functions.
        
    Methods
    -------
    register(name)
        Decorator to register a new conjugation rule under the specified name.
    get(name)
        Retrieve the conjugation rule function for the specified gate name.
        
    Notes
    -----
    When a rule is registered using the `register` decorator, it is also
    attached as a static method to the class with an uppercase name followed
    by "gate" (e.g., "CXgate" for "cx").
    
    Each registered rule should be a pure function that takes a PauliPath
    and relevant qubit indices as input, and returns a list of PauliPath
    objects representing the result after conjugation.
    
    Examples
    --------
    >>> @QuantumGate.register("new_gate")
    >>> def _new_gate_rule(path, qubit):
    ...     # Implementation of the rule
    ...     return [transformed_path]
    ...
    >>> # Can be accessed in two ways:
    >>> QuantumGate.get("new_gate")(path, 0)
    >>> QuantumGate.NEW_GATEgate(path, 0)
    """


    # {"cx": some_function, "t": another_function, ...}
    _registry: Dict[str, Callable] = {}

    # -------- registration decorator --------
    @classmethod
    def register(cls, name: str):
        """Decorator: register a new rule under *name* and
        attach it as <NAME>gate attribute for convenience."""
        def wrapper(func: Callable):
            # add name of gate and actual function of gate to class variable _registry
            cls._registry[name] = func 

            # it gives the cls a method named "CXgate" from "_cx_rule"
            setattr(cls, f"{name.upper()}gate", staticmethod(func)) 
            return staticmethod(func)
        return wrapper

    # -------- lookup helper --------
    @classmethod
    def get(cls, name: str) -> Callable:
        '''
        Search for if _registry contains a function associated with "name"
        If can't find it, raise an error
        '''
        try:
            return cls._registry[name]
        except KeyError as exc:
            raise NotImplementedError(f"No rule registered for gate '{name}'") from exc


# ---------- Implementation of CX and T gates ----------
# !!! 需要给这玩意写test
@QuantumGate.register("cx")
def _cx_rule(path: PauliPath, ctrl: int, tgt: int) -> List[PauliPath]:
    """
    Conjugate a Pauli operator by a controlled-X (CNOT) gate.
    
    This function implements the transformation rule for conjugating a Pauli
    operator by a CNOT gate: U†PU where U is the CNOT gate from control to target.
    
    The transformation follows these rules:
    - X on control: X_c -> X_c X_t
    - Z on control: Z_c -> Z_c
    - Y on control: Y_c -> Y_c X_t
    - X on target: X_t -> X_t
    - Z on target: Z_t -> Z_c Z_t
    - Y on target: Y_t -> Z_c Y_t
    
    Parameters
    ----------
    path : PauliPath
        The Pauli path to be transformed.
    ctrl : int
        Index of the control qubit.
    tgt : int
        Index of the target qubit.
    
    Returns
    -------
    List[PauliPath]
        A list containing a single PauliPath representing the transformed operator.
        
    Notes
    -----
    The implementation uses the binary representation of Pauli operators:
    - (z=0, x=0) : I (identity)
    - (z=0, x=1) : X
    - (z=1, x=0) : Z
    - (z=1, x=1) : Y = i*X*Z
    
    The XOR operation (^=) is used to toggle bits according to the transformation rules.
    """
    z, x = path.pauli.z.copy(), path.pauli.x.copy()

    # control qubit transforms
    if x[ctrl] and not z[ctrl]:        # Xc → Xc Xt
        # x[ctrl] = 1, z[ctrl] = 0, (z[ctrl], x[ctrl]) = (0,1), is X
        # X[tgt]
        x[tgt] ^= True  # ^= is XOR, 
    elif x[ctrl] and z[ctrl]:          # Yc → Yc Xt
        x[tgt] ^= True
    # Zc unchanged

    # target qubit transforms
    if x[tgt] and z[tgt]:              # Yt → Zc Yt
        z[ctrl] ^= True
    elif z[tgt] and not x[tgt]:        # Zt → Zc Zt
        z[ctrl] ^= True
    # Xt unchanged

    return [PauliPath(path.coeff, Pauli((z, x)))]


@QuantumGate.register("t")
def _t_rule(path: PauliPath, q: int) -> List[PauliPath]:
    """Conjugate a Pauli operator by a T gate (π/4 phase on |1⟩).
    
    This function implements the transformation rules for Pauli operators
    when conjugated by a T gate (T† P T). The T gate applies a π/4 phase
    to the |1⟩ state.
    
    The transformation rules are:
    - Z → Z (unchanged)
    - X → (X - Y)/√2
    - Y → (X + Y)/√2
    - I → I (unchanged)
    
    Parameters
    ----------
    path : PauliPath
        The Pauli path to be transformed.
    q : int
        Index of the qubit on which the T gate acts.
        
    Returns
    -------
    List[PauliPath]
        A list of PauliPath objects representing the transformed operator.
        For Z and I operators, returns a single-element list with the original path.
        For X and Y operators, returns a two-element list with the transformed paths.
        
    Notes
    -----
    The T gate is a single-qubit phase gate that applies a π/4 phase shift.
    It is represented by the matrix:
    T = [[1, 0], [0, exp(iπ/4)]]
    
    This implementation uses the binary representation of Pauli operators where
    each operator is encoded by two binary arrays (z, x).
    """
    z, x = path.pauli.z.copy(), path.pauli.x.copy()

    # Z  → unchanged
    if z[q] and not x[q]:
        return [path]

    # X  → (X − Y)/√2
    if x[q] and not z[q]:
        p1 = PauliPath(path.coeff / np.sqrt(2), path.pauli)            # +X
        z2 = z.copy(); z2[q] = True                                    # +Y
        p2 = PauliPath(-path.coeff / np.sqrt(2), Pauli((z2, x)))       # −Y
        return [p1, p2]

    # Y  → (X + Y)/√2
    if x[q] and z[q]:
        p1 = PauliPath(path.coeff / np.sqrt(2), path.pauli)            # +Y
        z2 = z.copy(); z2[q] = False                                   # +X
        p2 = PauliPath(path.coeff / np.sqrt(2), Pauli((z2, x)))        # +X
        return [p1, p2]

    # I on that qubit
    return [path]


Qiskit version: 2.0.0


In [19]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli

# 你的 3-层电路:
qc = QuantumCircuit(3, name="demo")
qc.cx(0, 1)          # layer-1
for q in range(3):   # layer-2
    qc.t(q)
qc.cx(1, 2)          # layer-3
qc.draw('text')


┌───┐     
q_0: ──■──┤ T ├─────
     ┌─┴─┐├───┤     
q_1: ┤ X ├┤ T ├──■──
     ├───┤└───┘┌─┴─┐
q_2: ┤ T ├─────┤ X ├
     └───┘     └───┘

In [20]:
q2i = {}
for i, q in enumerate(qc.qubits):
    q2i[q] = i
q2i

{<Qubit register=(3, "q"), index=0>: 0,
 <Qubit register=(3, "q"), index=1>: 1,
 <Qubit register=(3, "q"), index=2>: 2}

In [21]:
for d in qc.data:
    print(d)


CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=0>, <Qubit register=(3, "q"), index=1>), clbits=())
CircuitInstruction(operation=Instruction(name='t', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=0>,), clbits=())
CircuitInstruction(operation=Instruction(name='t', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=1>,), clbits=())
CircuitInstruction(operation=Instruction(name='t', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=2>,), clbits=())
CircuitInstruction(operation=Instruction(name='cx', num_qubits=2, num_clbits=0, params=[]), qubits=(<Qubit register=(3, "q"), index=1>, <Qubit register=(3, "q"), index=2>), clbits=())


In [7]:
# ─── 0. 准备：初始可观测量 ───
paths = [PauliPath(1.0, Pauli("IXI"))]   #  q2 q1 q0 = I X I
print("step 0  :", paths)                # [+1·IXI]



step 0  : [+1·IXI]


In [9]:
# ─── 1. 第一层（最右边）门：CX(control=1, target=2) ───
instr = qc.data[-1]                      # 最后一条指令
rule  = QuantumGate.get(instr.operation.name)
qidx  = [qc.qubits.index(q) for q in instr.qubits]   # [1, 2]

next_paths = []
for p in paths:
    next_paths.extend(rule(p, *qidx))    # 调用 _cx_rule
next_paths

[+1·IXX]

In [22]:
qidx

[1, 2]

In [14]:
instr.qubits

(<Qubit register=(3, "q"), index=1>, <Qubit register=(3, "q"), index=2>)

In [10]:
#   1-a  可选重量截断 (这里只有演示; 设 max_weight=None 就什么也不做)
max_weight = None
if max_weight is not None:
    next_paths = [p for p in next_paths
                  if (p.pauli.x | p.pauli.z).sum() <= max_weight]
    
next_paths

[+1·IXX]

In [12]:
#   1-b  合并同标签项
bucket = defaultdict(complex)
for p in next_paths:
    bucket[p.pauli.to_label()] += p.coeff
paths = [PauliPath(c, Pauli(lbl)) for lbl, c in bucket.items()]
paths

[+1+0j·IXX]

In [13]:
#   1-c  排序（权重升序，同权重时把正实系数的排前面）
paths.sort(key=lambda p: ((p.pauli.x | p.pauli.z).sum(),
                          p.coeff.real < 0))

print("step 1  after CX(1→2):", paths)   # [+1·XXI]

step 1  after CX(1→2): [+1+0j·IXX]


In [ ]:
# ── 2. 结果：最左端的 Pauli 展开 ────────────────────────────
final_paths = paths          # per_layer[-1] 也一样
print("\nfinal_paths :", final_paths)
# 预期打印：
#   [+0.707107·IXI  -0.707107·ZYI]

# ── 3. 单量子比特期望值（|+>⊗3） ────────────────────────────
plus_ev = {'I': 1.0, 'X': 1.0, 'Y': 0.0, 'Z': 0.0}

# ── 4. 手工累加 ⟨O⟩ ─────────────────────────────────────
exp_val = 0.0
for p in final_paths:
    term = p.coeff
    print(f"\nPath {p} :")
    for letter in p.label_lsb():            # 顺序 q0, q1, q2
        print(f"  × ⟨{letter}⟩", end="")
        term *= plus_ev.get(letter, 0.0)
        if term == 0.0:
            print(" = 0  (提前终止)")
            break
        else:
            print(f" = {term}")
    exp_val += term

print(f"\nExpectation  ⟨XXI⟩_|+>⊗3  with k=2  =  {exp_val}")

In [ ]:
# ------------------------
# 试验参数
# ------------------------


TRIALS   = 500          # 运行次数
TOLERANCE = 1e-12      # np.allclose 容差

failures = 0

_single = {"I": np.eye(2, dtype=complex),
           "X": np.array([[0, 1], [1, 0]], dtype=complex),
           "Y": np.array([[0, -1j], [1j, 0]], dtype=complex),
           "Z": np.array([[1, 0], [0, -1]], dtype=complex)}
    
def pauli_to_matrix(label: str) -> np.ndarray:
    """2-qubit label (big-endian) → 4×4 matrix."""
    return np.kron(_single[label[0]], _single[label[1]])

def series_to_matrix(series):
    """list[PauliTerm] → summed 4×4 matrix."""
    acc = np.zeros((4, 4), dtype=complex)
    for term in series:
        acc += term.coeff * pauli_to_matrix(term.pauli.to_label())
    return acc
LABELS_2Q = ["".join(p) for p in itertools.product("IXYZ", repeat=2)]
for _ in range(TRIALS):
    # --- 1. 随机 SU(4) 门 -------------------
    U = random_su4()
    gate = UnitaryGate(U, label="randSU4"); gate._name = "su4"
    
    # --- 2. Very small circuit --------------
    qc = QuantumCircuit(2)
    qc.append(gate, [0, 1])   # [高位, 低位] → big-endian
    
    # --- 3. 随机输入 Pauli -------------------
    label = np.random.choice(LABELS_2Q)
    P_in  = Pauli(label)
    
    # --- 4. Propagate ------------------------
    prop   = PauliPropagator(qc)
    series = prop.propagate(PauliTerm(1.0, P_in))[-1]   # list[PauliTerm]
    
    # --- 5. 矩阵对比 -------------------------
    lhs = U.conj().T @ pauli_to_matrix(label) @ U   # U† P U
    rhs = series_to_matrix(series)                  # Σ cᵢ Pᵢ
    
    if not np.allclose(lhs, rhs, atol=TOLERANCE):
        failures += 1

print(f"Total failures: {failures} / {TRIALS}")


In [ ]:


# --------------------------------------------------------------------
SYMS_STATE = "01+-rl"
SYMS_PAULI = "IXYZ"

def random_state_label(n: int) -> str:
    """Random product-state label, big-endian (left char = qubit-n-1)."""
    return "".join(random.choice(SYMS_STATE) for _ in range(n))


def random_pauli_label(n: int) -> str:
    """Random Pauli label with at least one non-I."""
    label = ["I"] * n
    idx = random.randrange(n)               # ensure one non-I
    label[idx] = random.choice("XYZ")
    for i in range(n):
        if i != idx:
            label[i] = random.choice(SYMS_PAULI)
    return "".join(label)

def build_random_circuit(n: int, gate_count: int) -> QuantumCircuit:
    """Generate a random circuit containing T, CX and SU(4) gates."""
    qc = QuantumCircuit(n, name=f"rand_mix_{n}q_{gate_count}g")
    for _ in range(gate_count):
        gtype = random.choice(("t", "cx", "su4"))
        if gtype == "t":
            qc.t(random.randrange(n))
        elif gtype == "cx":
            ctrl, tgt = random.sample(range(n), 2)
            qc.cx(ctrl, tgt)
        else:                               # SU(4)
            q1, q2 = random.sample(range(n), 2)
            gate = UnitaryGate(random_su4(), label="randSU4")
            gate._name = "su4"              # dispatch → _su4_rule
            qc.append(gate, [q1, q2])
    return qc

# --------------------------------------------------------------------
TRIALS = 100
TOL     = 1e-10

for _ in tqdm(range(TRIALS)):
    """Compare PauliPropagator vs exact state-vector for random circuits."""
    n          = random.randint(3, 10)
    gate_count = random.randint(5, 10)
    qc         = build_random_circuit(n, gate_count)

    state_lbl  = random_state_label(n)
    pauli_lbl  = random_pauli_label(n)
    observable = Pauli(pauli_lbl)

    # --- PauliPropagator expectation ---------------------------------
    prop = PauliPropagator(qc)
    layers   = prop.propagate(PauliTerm(1.0, observable))
    prop_ev  = prop.expectation_pauli_sum(layers[-1], state_lbl)

    # --- Full state-vector expectation -------------------------------
    sv_ev = Statevector.from_label(state_lbl).evolve(qc).expectation_value(
        SparsePauliOp.from_list([(pauli_lbl, 1.0)])).real

    print(abs(prop_ev - sv_ev) < TOL)

In [ ]:

# @QuantumGate.register("su4")
# def _su4_rule(term: PauliTerm,
#               q1: int,
#               q2: int,
#               mat: np.ndarray) -> List[PauliTerm]:
#     """
#     Conjugate a PauliTerm by an SU(4) unitary acting on qubits q1 and q2
#     (big-endian: right-most label char is qubit-0).
#     """
#     LABELS_2Q = (
#         "II","IX","IY","IZ",
#         "XI","XX","XY","XZ",
#         "YI","YX","YY","YZ",
#         "ZI","ZX","ZY","ZZ"
#     )

#     @lru_cache(maxsize=None)
#     def _pauli_matrix(label: str) -> np.ndarray:
#         """Return the 4脳4 matrix of a two-qubit Pauli label."""
#         _single = {
#             "I": np.eye(2, dtype=complex),
#             "X": np.array([[0, 1], [1, 0]], dtype=complex),
#             "Y": np.array([[0, -1j], [1j, 0]], dtype=complex),
#             "Z": np.array([[1, 0], [0, -1]], dtype=complex)
#         }
#         return np.kron(_single[label[0]], _single[label[1]])

#     # 1) extract the two-qubit substring on (q1, q2)
#     full_label = list(term.pauli.to_label())          # big-endian
#     two_label  = full_label[-1 - q2] + full_label[-1 - q1]

#     # 2) build / fetch transfer matrix R
#     # --- NEW robust cache key: matrix bytes + qubit ordering -----------
#     cache_key = (mat.tobytes(), q1, q2)
#     # -------------------------------------------------------------------
#     if not hasattr(_su4_rule, "_cache"):
#         _su4_rule._cache = {}
#     if cache_key not in _su4_rule._cache:
#         R = np.zeros((16, 16), dtype=complex)
#         mat_dag = mat.conj().T
#         for beta, P_beta in enumerate(LABELS_2Q):
#             conj = mat_dag @ _pauli_matrix(P_beta) @ mat
#             for alpha, P_alpha in enumerate(LABELS_2Q):
#                 R[alpha, beta] = 0.25 * np.trace(_pauli_matrix(P_alpha) @ conj)
#         _su4_rule._cache[cache_key] = R
#     else:
#         R = _su4_rule._cache[cache_key]

#     # 3) column 尾 gives new coefficients
#     beta_idx = LABELS_2Q.index(two_label)
#     coeffs   = R[:, beta_idx] * term.coeff

#     # 4) assemble new PauliTerm list
#     terms: List[PauliTerm] = []
#     for alpha_idx, c in enumerate(coeffs):
#         if abs(c) < 1e-12:
#             continue
#         alpha_label = LABELS_2Q[alpha_idx]
#         new_label = full_label.copy()
#         new_label[-1 - q2] = alpha_label[0]   # 伪鈧€ 鈫� q2
#         new_label[-1 - q1] = alpha_label[1]   # 伪鈧� 鈫� q1
#         terms.append(PauliTerm(c, Pauli("".join(new_label))))
#     return terms


In [ ]:


# class PauliPropagator:
#     """
#     Back-propagate a Pauli observable through a QuantumCircuit
#     using purely integer bit-masks internally.
#     Only CX / T rules are currently implemented.
#     """
#     _SINGLE_Q_EV = {  # expectation <state|P|state> for product states
#         "0": {"I":1, "X":0, "Y":0, "Z": 1},
#         "1": {"I":1, "X":0, "Y":0, "Z":-1},
#         "+": {"I":1, "X":1, "Y":0, "Z": 0},
#         "-": {"I":1, "X":-1,"Y":0, "Z": 0},
#         "r": {"I":1, "X":0, "Y":1, "Z": 0},
#         "l": {"I":1, "X":0, "Y":-1,"Z": 0},
#     }

#     def __init__(self, qc: QuantumCircuit):
#         self.qc   = qc
#         self.n    = qc.num_qubits
#         self.q2i  = {q: i for i, q in enumerate(qc.qubits)}

# # %% ---------- PauliPropagator.propagate  全量替换 ----------
#     # def propagate(
#     #     self,
#     #     observable: PauliTerm,
#     #     max_weight: int | None = None,
#     #     tol: float = 1e-12,
#     # ) -> List[List[PauliTerm]]:

#     #     if observable.n != self.n:
#     #         raise ValueError("Observable qubit count mismatch")

#     #     paths: Dict[int, complex] = {observable.key: observable.coeff}
#     #     history: List[List[PauliTerm]] = [[observable]]

#     #     # —— 逆序遍历所有门 ——
#     #     for instr in reversed(self.qc.data):
#     #         gname = instr.operation.name
#     #         rule  = QuantumGate.get(gname)
#     #         qidx  = tuple(self.q2i[q] for q in instr.qubits)

#     #         extra_args = ()
#     #         if gname == "su4" and hasattr(instr.operation, "to_matrix"):
#     #             extra_args = (instr.operation.to_matrix().astype(np.complex128),)

#     #         next_paths: Dict[int, complex] = {}

#     #         # —— 对当前所有路径应用该门 ——
#     #         for key_in, coeff_in in paths.items():
#     #             L, coeff_arr, key_arr = rule(coeff_in, key_in, self.n, *qidx, *extra_args)

#     #             for j in range(int(L)):
#     #                 k2 = int(key_arr[j])
#     #                 c2 = coeff_arr[j]

#     #                 if max_weight is not None and \
#     #                    weight_of_key(k2, self.n) > max_weight:
#     #                     continue
#     #                 next_paths[k2] = next_paths.get(k2, 0.0) + c2

#     #         # —— 剪掉 |coeff| < tol ——
#     #         paths = {k: c for k, c in next_paths.items() if abs(c) > tol}

#     #         # —— 收录本层 —— 
#     #         history.append([PauliTerm(c, k, self.n) for k, c in paths.items()])

#     #     return history

#     # --------------------------------------------------
#     def propagate(
#         self,
#         observable: PauliTerm,
#         max_weight: int | None = None,
#         tol: float = 1e-12,
#     ) -> List[List[PauliTerm]]:
#         """
#         反向传播 *observable* 至电路输入端，返回逐层历史。
#         依赖：各门规则已在 QuantumGate._registry 中注册为
#               Numba CPUDispatcher，且统一返回
#               (L:int, coeff_arr:complex128[::1], key_arr:int64[::1])
#         """
#         if observable.n != self.n:
#             raise ValueError("Observable qubit count mismatch")

#         # ── 当前路径：{ key(int掩码) : coeff(complex) } ──
#         paths: dict[int, complex] = {observable.key: observable.coeff}
#         history: List[List[PauliTerm]] = [[observable]]

#         # ── 逆序遍历电路 ────────────────────────────────
#         for instr in reversed(self.qc.data):
#             gname = instr.operation.name
#             rule  = QuantumGate.get(gname)
#             qidx  = tuple(self.q2i[q] for q in instr.qubits)

#             extra_args = ()
#             if gname == "su4" and hasattr(instr.operation, "to_matrix"):
#                 # Numba 内核要求 np.complex128 且 C-contiguous
#                 extra_args = (instr.operation.to_matrix().astype(np.complex128),)

#             next_paths: dict[int, complex] = {}

#             # —— 对 paths 中的每条项调用一次 numba kernel ——
#             for key_in, coeff_in in paths.items():
#                 L, coeff_arr, key_arr = rule(
#                     coeff_in, key_in, self.n, *qidx, *extra_args
#                 )

#                 for j in range(int(L)):
#                     k2 = int(key_arr[j])
#                     c2 = coeff_arr[j]

#                     if max_weight is not None and \
#                        weight_of_key(k2, self.n) > max_weight:
#                         continue
#                     next_paths[k2] = next_paths.get(k2, 0.0) + c2

#             # —— 剪掉系数极小的路径 —— 
#             paths = {k: c for k, c in next_paths.items() if abs(c) > tol}

#             # —— 保存本层 —— 
#             history.append([PauliTerm(c, k, self.n) for k, c in paths.items()])

#         return history


#     def expectation_pauli_sum(self,
#                               pauli_sum: List[PauliTerm],
#                               product_label: str) -> float:
#         """
#         Compute sum_i alpha_i * Tr[rho * P_i] for a product computational/basis state.
#         """
#         if len(product_label) != self.n:
#             raise ValueError("Product-state label length mismatch")

#         total = 0.0
#         for term in pauli_sum:
#             pauli_str = term.to_label()
#             factor = 1.0
#             for qubit_index in range(self.n):
#                 letter = pauli_str[self.n - 1 - qubit_index]
#                 state_symbol = product_label[self.n - 1 - qubit_index]
#                 factor *= self._SINGLE_Q_EV[state_symbol][letter]
#                 if factor == 0.0:
#                     break
#             total += term.coeff * factor
#         return float(total.real)

In [ ]:
# # %%
# # ---------- Z‖X  <->  int ----------
# def encode_pauli(p: Pauli) -> int:
#     n = len(p.z)
#     x_bits = 0
#     z_bits = 0
#     for i in range(n):
#         if p.x[i]:
#             x_bits |= 1 << i
#         if p.z[i]:
#             z_bits |= 1 << i
#     return (z_bits << n) | x_bits

# def decode_pauli(key: int, n: int) -> Pauli:
#     mask   = (1 << n) - 1
#     x_bits =  key        & mask
#     z_bits = (key >> n)  & mask
#     x = np.array([(x_bits >> i) & 1 for i in range(n)], dtype=bool)
#     z = np.array([(z_bits >> i) & 1 for i in range(n)], dtype=bool)
#     return Pauli((z, x))

# @njit(cache=True)
# def _weight_of_key_nb(key: int, n: int) -> int:
#     mask = (1 << n) - 1
#     bits = (key & mask) | (key >> n)
#     cnt  = 0
#     while bits:
#         bits &= bits - 1
#         cnt  += 1
#     return cnt

# def weight_of_key(key: int, n: int) -> int:
#     return _weight_of_key_nb(key, n)
# # %%
# # ───────────────────────
# #  PauliTerm：coeff + key
# # ───────────────────────
# @dataclass(slots=True)
# class PauliTerm:
#     coeff: complex
#     key:   int
#     n:     int

#     def to_label(self) -> str:
#         return decode_pauli(self.key, self.n).to_label()

#     def __repr__(self) -> str:
#         return f"{self.coeff:+g}·{self.to_label()}"

# # %%
# # ─────────────────────────────
# #  QuantumGate registry  (直接存 numba dispatcher)
# # ─────────────────────────────
# class QuantumGate:
#     _registry: Dict[str, callable] = {}

#     @classmethod
#     def get(cls, name: str):
#         if name not in cls._registry:
#             raise NotImplementedError(f"No rule for gate '{name}'")
#         return cls._registry[name]

# # %%
# # ─────────────────────────────
# #  Bit-kernels：CX / T → 统一返回 5-tuple
# # ─────────────────────────────

# @njit(cache=True)
# def _cx_bits_nb(coeff: complex, key: int, n: int,
#                 ctrl: int, tgt: int) -> Tuple[int, complex, int, complex, int]:
#     # extract
#     x_c = (key >>  ctrl)     & 1
#     z_c = (key >> (n+ctrl))  & 1
#     x_t = (key >>  tgt)      & 1
#     z_t = (key >> (n+tgt))   & 1
#     # phase
#     minus = x_c & z_t & (1 ^ (x_t ^ z_c))
#     phase = -1 if minus else +1
#     # update bits
#     x_tn = x_t ^ x_c
#     z_cn = z_c ^ z_t
#     outk = key
#     if x_tn != x_t:
#         outk ^= 1 << tgt
#     if z_cn != z_c:
#         outk ^= 1 << (n+ctrl)
#     # 返回 (长度, c1, k1, c2, k2)
#     return 1, coeff * phase, outk, 0.0+0.0j, 0

# @njit(cache=True)
# def _t_bits_nb(coeff: complex, key: int, n: int, q: int) -> Tuple[int, complex, int, complex, int]:
#     x = (key >>  q)     & 1
#     z = (key >> (n+q))  & 1
#     # I or Z unchanged
#     if (z and not x) or (not x and not z):
#         return 1, coeff, key, 0.0+0.0j, 0
#     # X/Y case
#     key2 = key ^ (1 << (n+q))
#     c1   = coeff / np.sqrt(2)
#     c2   = +c1 if z else -c1
#     return 2, c1, key, c2, key2
# # %%
# import numpy as np
# from numba import njit, int64, complex128, float64
# from typing import List, Dict, Tuple
# from qiskit import QuantumCircuit

# # weight_of_key 必须已在别处定义（位掩码 popcount）——————————
# # def weight_of_key(key:int, n:int) -> int: ...

# # QuantumGate 必须已注册好 cx / t / su4 三个 numba 内核 ————————

# # %%
# class PauliPropagator:
#     """
#     纯掩码版 Pauli 反向传播器：
#       * propagate()   — 依赖已注册的 numba 内核 (CX/T/SU4)
#       * expectation_pauli_sum() — 纯 numba 查表
#     """

#         # --------------------------------------------------
#     def __init__(self, qc: QuantumCircuit):
#         self.qc  = qc
#         self.n   = qc.num_qubits
#         self.q2i = {q: i for i, q in enumerate(qc.qubits)}


#     # ---------- 单量子比特查表 (state_idx × z × x) ----------
#     _STATE_IDX = {'0':0,'1':1,'+':2,'-':3,'r':4,'l':5}
#     _EXP_TABLE = np.zeros((6,2,2), dtype=np.float64)
#     _EXP_TABLE[_STATE_IDX['0'],:] = [[1,0],[1,0]]
#     _EXP_TABLE[_STATE_IDX['1'],:] = [[1,0],[-1,0]]
#     _EXP_TABLE[_STATE_IDX['+'],:] = [[1,1],[0,0]]
#     _EXP_TABLE[_STATE_IDX['-'],:] = [[1,-1],[0,0]]
#     _EXP_TABLE[_STATE_IDX['r'],:] = [[1,0],[0,1]]
#     _EXP_TABLE[_STATE_IDX['l'],:] = [[1,0],[0,-1]]

#     # ---------- expectation kernel ----------
#     @staticmethod
#     @njit(cache=True)
#     def _expect_keys(coeffs: complex128[:], keys: int64[:],
#                      state_idxs: int64[:], n: int64,
#                      exp_table: float64[:, :, :]) -> float64:
#         total = 0.0
#         ns    = coeffs.size
#         for t in range(ns):
#             key   = keys[t]
#             alpha = coeffs[t]
#             prod  = 1.0
#             for q in range(n):
#                 x = (key >>  q)     & 1
#                 z = (key >> (n+q))  & 1
#                 prod *= exp_table[state_idxs[q], z, x]
#                 if prod == 0.0:
#                     break
#             total += alpha.real * prod     # 结果保证实数
#         return total


#     # --------------------------------------------------
#     def propagate(self,
#                   observable: 'PauliTerm',
#                   max_weight: int | None = None,
#                   tol: float = 1e-12
#                  ) -> List[List['PauliTerm']]:

#         if observable.n != self.n:
#             raise ValueError("Observable qubit count mismatch")

#         paths: Dict[int, complex] = {observable.key: observable.coeff}
#         history: List[List['PauliTerm']] = [[observable]]

#         for instr in reversed(self.qc.data):
#             gname = instr.operation.name
#             rule  = QuantumGate.get(gname)
#             qidx  = tuple(self.q2i[q] for q in instr.qubits)

#             extra_args: Tuple = ()
#             if gname == "su4" and hasattr(instr.operation, "to_matrix"):
#                 extra_args = (instr.operation.to_matrix().astype(np.complex128),)

#             next_paths: Dict[int, complex] = {}

#             for key_in, coeff_in in paths.items():
#                 L, coeff_arr, key_arr = rule(coeff_in, key_in,
#                                              self.n, *qidx, *extra_args)

#                 for j in range(int(L)):
#                     k2 = int(key_arr[j])
#                     c2 = coeff_arr[j]
#                     if max_weight is not None and \
#                        weight_of_key(k2, self.n) > max_weight:
#                         continue
#                     next_paths[k2] = next_paths.get(k2, 0.0) + c2

#             # 丢弃 |coeff| < tol
#             paths = {k: c for k, c in next_paths.items() if abs(c) > tol}

#             history.append([PauliTerm(c, k, self.n) for k, c in paths.items()])

#         return history

#     # --------------------------------------------------
#     def expectation_pauli_sum(self,
#                               pauli_sum: List['PauliTerm'],
#                               product_label: str) -> float:
#         """
#         高速计算 Σ_i α_i Tr[ρ P_i]，ρ 为 |product_label⟩ 的直积态。
#         """
#         if len(product_label) != self.n:
#             raise ValueError("Label length mismatch")

#         # |state⟩ → state_idx 数组 (LSB 对应 label[-1])
#         state_idxs = np.fromiter(
#             (self._STATE_IDX[ch] for ch in product_label[::-1]),
#             dtype=np.int64, count=self.n)

#         m = len(pauli_sum)
#         coeffs = np.empty(m, dtype=np.complex128)
#         keys   = np.empty(m, dtype=np.int64)
#         for i, term in enumerate(pauli_sum):
#             coeffs[i] = term.coeff
#             keys[i]   = term.key

#         return float(self._expect_keys(coeffs, keys,
#                                        state_idxs, self.n,
#                                        self._EXP_TABLE))


# # code: 0=I, 1=X, 2=Y, 3=Z
# _SINGLE_P = (
#     np.eye(2, dtype=np.complex128),
#     np.array([[0,1],[1,0]],      dtype=np.complex128),      # X
#     np.array([[0,-1j],[1j,0]],   dtype=np.complex128),      # Y
#     np.array([[1,0],[0,-1]],     dtype=np.complex128),      # Z
# )
# # P_STACK[idx]  idx = 4*code(q2)+code(q1)
# _P_STACK = np.stack([
#     np.kron(_SINGLE_P[q2], _SINGLE_P[q1])
#     for q2 in range(4) for q1 in range(4)
# ])   # shape (16,4,4)

# @njit(cache=True)
# def _code_from_bits(z: int64, x: int64) -> int64:
#     """(z,x) → 0(I),1(X),2(Y),3(Z)"""
#     #   z x | code
#     #   0 0 | 0
#     #   0 1 | 1
#     #   1 1 | 2
#     #   1 0 | 3
#     return (z << 1) | x if z == 0 else (2 | (x ^ 1))

# @njit(cache=True)
# def _bits_from_code(code: int64) -> tuple:    # 返回 (z,x)
#     if code == 0:   return 0,0
#     elif code == 1: return 0,1
#     elif code == 2: return 1,1
#     else:           return 1,0

# # %% ---------- SU(4) 共轭规则  (Numba) ----------
# @njit(cache=True)
# def _su4_bits_nb(coeff: complex128, key: int64, n: int64,
#                  q1: int64, q2: int64, mat: np.ndarray
#                 ) -> tuple:
#     """
#     返回三元：(L, coeff_arr[complex128], key_arr[int64])
#        L ≤ 16    （当某些系数≈0 被跳过时更小）
#     """
#     # ——— 1. 提取 β = P[q2,q1] 代码 ———
#     x1 = (key >>  q1)     & 1
#     z1 = (key >> (n+q1))  & 1
#     x2 = (key >>  q2)     & 1
#     z2 = (key >> (n+q2))  & 1
#     code1 = _code_from_bits(z1, x1)           # q1
#     code2 = _code_from_bits(z2, x2)           # q2
#     beta_idx = 4*code2 + code1

#     # ——— 2. 计算 U† P U ———
#     conj = mat.conj().T @ _P_STACK[beta_idx] @ mat
#     coeffs = 0.25 * np.array([np.trace(_P_STACK[i].conj().T @ conj)
#                               for i in range(16)], dtype=np.complex128)

#     # ——— 3. 写出有效 (coeff,key) 列表 ———
#     coeff_out = np.empty(16, dtype=np.complex128)
#     key_out   = np.empty(16, dtype=np.int64)
#     L = 0
#     for alpha in range(16):
#         c = coeff * coeffs[alpha]
#         if abs(c.real)+abs(c.imag) < 1e-12:
#             continue
#         # derive (z,x) for q2,q1 from alpha
#         code2a = alpha // 4
#         code1a = alpha  & 3

#         new_key = key
#         # update q1
#         z,x = _bits_from_code(code1a)
#         if ((new_key >>  q1) & 1)     != x: new_key ^= 1 <<  q1
#         if ((new_key >> (n+q1)) & 1) != z: new_key ^= 1 << (n+q1)
#         # update q2
#         z,x = _bits_from_code(code2a)
#         if ((new_key >>  q2) & 1)     != x: new_key ^= 1 <<  q2
#         if ((new_key >> (n+q2)) & 1) != z: new_key ^= 1 << (n+q2)

#         coeff_out[L] = c
#         key_out[L]   = new_key
#         L += 1

#     return L, coeff_out[:L], key_out[:L]


# # 注册到 QuantumGate

# QuantumGate._registry["cx"] = _cx_bits_nb
# setattr(QuantumGate, "CXgate", staticmethod(_cx_bits_nb))

# QuantumGate._registry["t"]  = _t_bits_nb
# setattr(QuantumGate, "Tgate",  staticmethod(_t_bits_nb))

# QuantumGate._registry["su4"] = _su4_bits_nb
# setattr(QuantumGate, "SU4gate", staticmethod(_su4_bits_nb))

In [ ]:

# # pauli_pkg/tests/test_gates.py

# # Exhaustive and analytical checks for CX / T conjugation rules.

# # No state-vector sims here: we compare the matrices from the rule against
# # U闁炽儻鎷� P U directly.


# import itertools, numpy as np, pytest
# from qiskit import QuantumCircuit
# from qiskit.quantum_info import Pauli, Operator
# from pauli_propagation import PauliTerm, QuantumGate

# LABELS_2Q = ["".join(p) for p in itertools.product("IXYZ", repeat=2)]

# def pauli_matrix(lbl):           # helper
#     return Pauli(lbl).to_matrix()

# def cx_matrix(ctrl, tgt):
#     qc = QuantumCircuit(2)
#     qc.cx(ctrl, tgt)
#     return Operator(qc).data

# def sum_paths(paths):
#     return sum(p.coeff * p.pauli.to_matrix() for p in paths)

# @pytest.mark.parametrize("ctrl,tgt", [(0,1), (1,0)])
# @pytest.mark.parametrize("label", LABELS_2Q)
# def test_cx_rule(ctrl, tgt, label):
#     U  = cx_matrix(ctrl, tgt)
#     inp = PauliTerm(1.0, Pauli(label))
#     out = QuantumGate.get("cx")(inp, ctrl, tgt)
#     assert np.allclose(sum_paths(out), U.conj().T @ pauli_matrix(label) @ U)

# @pytest.mark.parametrize("label", list("IXYZ"))
# def test_t_single(label):
#     from math import pi
#     T = np.diag([1.0, np.exp(1j*pi/4)])
#     inp = PauliTerm(1.0, Pauli(label))
#     out = QuantumGate.get("t")(inp, 0)
#     assert np.allclose(sum_paths(out), T.conj().T @ Pauli(label).to_matrix() @ T)

# def test_t_random_embedded():
#     from math import pi
#     import random
#     T = np.diag([1.0, np.exp(1j*pi/4)])

#     for _ in range(100):
#         label4 = "".join(random.choice("IXYZ") for _ in range(4))
#         q      = random.randrange(4)
#         inp    = PauliTerm(1.0, Pauli(label4))
#         out    = QuantumGate.get("t")(inp, q)

#         # build reference matrix 闁�?閿�?-wise (little-endian)
#         mats = []
#         for idx, ch in enumerate(reversed(label4)):
#             base = Pauli(ch).to_matrix()
#             mats.append(T.conj().T @ base @ T if idx == q else base)

#         ref = mats[0]
#         for m in mats[1:]:
#             ref = np.kron(m, ref)

#         assert np.allclose(sum_paths(out), ref)